### SPAM Ham Detection

In [1]:
import random
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
## Reading the given dataset
spam = pd.read_csv("SMSSpamCollection.txt", sep = "\t", names=["label", "message"])

In [3]:
print(spam.head())

  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [4]:
## Converting the read dataset in to a list of tuples, each tuple(row) contianing the message and it's label
data_set = []
for index,row in spam.iterrows():
    data_set.append((row['message'], row['label']))

In [5]:
print(data_set[:5])

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'ham'), ('Ok lar... Joking wif u oni...', 'ham'), ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'spam'), ('U dun say so early hor... U c already then say...', 'ham'), ("Nah I don't think he goes to usf, he lives around here though", 'ham')]


In [6]:
print(len(data_set))

5572


### Preprocessing

In [7]:
## initialise the inbuilt Stemmer and the Lemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess(document, stem=True):
    'changes document to lower case, removes stopwords and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)

    return document

In [9]:
## - Performing the preprocessing steps on all messages
messages_set = []
for (message, label) in data_set:
    words_filtered = [e.lower() for e in preprocess(message, stem=False).split() if len(e) >= 3]
    messages_set.append((words_filtered, label))

In [10]:
print(messages_set[:5])

[(['jurong', 'point', 'crazy', 'available', 'bugis', 'great', 'world', 'buffet', '...', 'cine', 'get', 'amore', 'wat', '...'], 'ham'), (['lar', '...', 'joke', 'wif', 'oni', '...'], 'ham'), (['free', 'entry', 'wkly', 'comp', 'win', 'cup', 'final', 'tkts', '21st', 'may', '2005.', 'text', '87121', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 'apply', '08452810075over18'], 'spam'), (['dun', 'say', 'early', 'hor', '...', 'already', 'say', '...'], 'ham'), (['nah', "n't", 'think', 'usf', 'live', 'around', 'though'], 'ham')]


### Preparing to create features

In [11]:
## - creating a single list of all words in the entire dataset for feature list creation

def get_words_in_messages(messages):
    all_words = []
    for (message, label) in messages:
      all_words.extend(message)
    return all_words

In [12]:
## - creating a final feature list using an intuitive FreqDist, to eliminate all the duplicate words
## Note : we can use the Frequency Distribution of the entire dataset to calculate Tf-Idf scores like we did earlier.

def get_word_features(wordlist):

    #print(wordlist[:10])
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [13]:
## - creating the word features for the entire dataset
word_features = get_word_features(get_words_in_messages(messages_set))
print(len(word_features))

7995


### Preparing to create a train and test set

In [14]:
## - creating slicing index at 80% threshold
sliceIndex = int((len(messages_set)*.8))

In [15]:
## - shuffle the pack to create a random and unbiased split of the dataset
random.shuffle(messages_set)

In [16]:
train_messages, test_messages = messages_set[:sliceIndex], messages_set[sliceIndex:]

In [18]:
print(len(train_messages))
print(len(test_messages))

4457
1115


### Preparing to create feature maps for train and test data

In [19]:
## creating a LazyMap of feature presence for each of the 8K+ features with respect to each of the SMS messages
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [20]:
## - creating the feature map of train and test data

training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features, test_messages)

In [21]:
print(training_set[:5])

contains(phonebook)': False, 'contains(ash)': False, 'contains(shijutta)': False, 'contains(offense)': False, 'contains(dvg)': False, 'contains(vinobanagar)': False, 'contains(ovulate.when)': False, 'contains(3wks)': False, 'contains(woah)': False, 'contains(orh)': False, 'contains(secrets)': False, 'contains(soon.xxx)': False, 'contains(cool-mob)': False, 'contains(frog-axel)': False, 'contains(akon-lonely)': False, 'contains(eyed-dont)': False, 'contains(cashbin.co.uk)': False, 'contains(www.cashbin.co.uk)': False, 'contains(canteen)': False, 'contains(stressfull)': False, 'contains(president)': False, 'contains(140)': False, 'contains(180)': False, 'contains(bedrm)': False, 'contains(pleasured)': False, 'contains(support.providing)': False, 'contains(assistance)': False, 'contains(whens)': False, 'contains(1172.)': False, 'contains(lonlines)': False, 'contains(lotz)': False, 'contains(memories)': False, 'contains(gailxx)': False, 'contains(complacent)': False, 'contains(denis)': Fal

In [22]:
print('Training set size : ', len(training_set))
print('Test set size : ', len(testing_set))

Training set size :  4457
Test set size :  1115


### Training

In [23]:
## Training the classifier with NaiveBayes algorithm
spamClassifier = nltk.NaiveBayesClassifier.train(training_set)

### Evaluation

In [24]:
## - Analyzing the accuracy of the test set
print(nltk.classify.accuracy(spamClassifier, training_set))

0.9921471842046219


In [25]:
## Analyzing the accuracy of the test set
print(nltk.classify.accuracy(spamClassifier, testing_set))

0.9829596412556054


In [26]:
## Testing a example message with our newly trained classifier
m = 'CONGRATULATIONS!! As a valued account holder you have been selected to receive a £900 prize reward! Valid 12 hours only.'
print('Classification result : ', spamClassifier.classify(extract_features(m.split())))

Classification result :  spam


In [27]:
## Priting the most informative features in the classifier
print(spamClassifier.show_most_informative_features(50))

Most Informative Features
         contains(award) = True             spam : ham    =    202.1 : 1.0
        contains(urgent) = True             spam : ham    =    131.5 : 1.0
         contains(nokia) = True             spam : ham    =    105.9 : 1.0
         contains(await) = True             spam : ham    =    100.0 : 1.0
          contains(code) = True             spam : ham    =    100.0 : 1.0
        contains(camera) = True             spam : ham    =     91.5 : 1.0
          contains(rate) = True             spam : ham    =     83.0 : 1.0
         contains(video) = True             spam : ham    =     83.0 : 1.0
       contains(attempt) = True             spam : ham    =     78.7 : 1.0
        contains(expire) = True             spam : ham    =     70.2 : 1.0
       contains(private) = True             spam : ham    =     70.2 : 1.0
       contains(service) = True             spam : ham    =     68.8 : 1.0
      contains(landline) = True             spam : ham    =     67.7 : 1.0

In [28]:
## storing the classifier on disk for later usage
import pickle
f = open('nb_spam_classifier.pickle', 'wb')
pickle.dump(spamClassifier,f)
print('Classifier stored at ', f.name)
f.close()

Classifier stored at  nb_spam_classifier.pickle
